# Generador Reporte

Paso final de generación de reportes para la Encuesta de Caracterización Estudiantil 2023, este script toma la portada, realizarda previamente y que se encuentra en el directorio "portadas", el cual es un archivo jpg que es colocado como primera imagen para luego tomar el resto de las imágenes consistentes en los distintos gráficos, al finalizar crea un archivo PDF.

Se utiliza la librería FPDF para la generación de archivos PDF

Este script debe correrse como último paso.

In [29]:
#!pip install natsort

In [50]:
import glob
from fpdf import FPDF
from natsort import natsorted
import os
import re

In [64]:
def capit(name):
    frase_capitalizada = re.sub(r"\b(?!en)\w", lambda m: m.group().capitalize(), name)
    return frase_capitalizada

In [67]:
# Configuración de FPDF

class PDF(FPDF):
    def __init__(self, carrera):
        super().__init__()
        self.carrera = carrera
        self.page_number = 0  # Inicializa el número de página en 0

    def header(self):
        # Configuración del encabezado
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, "Encuesta de Caracterización 2023 - " + self.carrera, 0, 1, "C")

    def footer(self):
        if self.page_number != 1:  # Aquí evalua si es la primera página, para no agregar numeración
            # Configuración del pie de página
            self.set_y(-15)
            self.set_font("Arial", "I", 8)
            self.cell(0, 10, f"Página {self.page_no()}", 0, 0, "C")

    def add_page(self):
        super().add_page()
        self.page_number += 1

In [68]:
def agregar_portada(pdf):
    pdf.add_page()
    pdf.image('./portadas/portada.jpg', x=0, y=0, w=pdf.w, h=pdf.h)

In [76]:
def generar_reporte(carrera, ruta_base, subdirectorio):
    pdf = PDF(carrera)
    pdf.set_auto_page_break(auto=True, margin=15)
    
    agregar_portada(pdf)

    # Configuración de las imágenes
    imagen_width = 80
    imagen_height = 60
    margen_horizontal = 10
    margen_vertical = 10
    contador = 0
    imagenes_por_pagina = 6

    # Obtener la lista de archivos de imagen en la carpeta
    ruta = os.path.join(subdirectorio, f"{carrera}/*.png")
    imagenes = natsorted(glob.glob(ruta)) 

    for i, imagen in enumerate(imagenes):
        if contador % imagenes_por_pagina == 0:
            pdf.add_page()

        if contador % 2 == 0:
            pdf.ln(margen_vertical)

        x = margen_horizontal + (contador % 2) * (imagen_width + margen_horizontal)
        y = pdf.get_y() + 10  # Ajusta la coordenada Y para que esté debajo de la portada

        pdf.image(imagen, x, y, imagen_width, imagen_height)
        pdf.cell(imagen_width, 10, ln=False)

        if contador % 2 == 1:
            pdf.ln(imagen_height + margen_vertical)

        contador += 1
        
    nombre_archivo = os.path.join(ruta_base, f"reporte_{carrera.replace(' ', '_')}.pdf")
    pdf.output(nombre_archivo)

In [77]:
# Ruta de la imagen de portada
portada_directory = 'portadas'

## Generar informes

Se puede generar utilizando la función "generar_reporte" y ambos chunks deben correrse para crear los reportes tanto por carrera o el general por sede

In [79]:
# Generador informes por Carrera
for nombre in os.listdir('carreras'):
    nombre_carrera = capit(nombre.lower())
    generar_reporte(nombre_carrera, 'reportes', 'carreras')

In [80]:
# Generador informes por sede
for nombre in os.listdir('sede'):
    nombre_carrera = capit(nombre.lower())
    generar_reporte(nombre_carrera, 'reportes_sede', 'sede')